In [26]:
import pandas as pd
import openai
from openai.embeddings_utils import get_embedding
from datasets import Dataset
import numpy as np

In [27]:
openai.api_key = open("key.txt", "r").read()

In [28]:
embedding_model = "text-embedding-ada-002"

In [29]:
## create a list

semantic = ['machine learning has mainly two approaches supervised learning as well as unsupervised learning','in supervised learning we have input as well as output data','in unsupervised learning we have only input data','in supervised learning we have classification as well as regression','in unsupervised learning we have clustering as well as association','delhi is captial of india']
semantic

['machine learning has mainly two approaches supervised learning as well as unsupervised learning',
 'in supervised learning we have input as well as output data',
 'in unsupervised learning we have only input data',
 'in supervised learning we have classification as well as regression',
 'in unsupervised learning we have clustering as well as association',
 'delhi is captial of india']

In [30]:
df = pd.DataFrame(semantic,columns=['text'])
df

,text
0,machine learning has mainly two approaches sup...
1,in supervised learning we have input as well a...
2,in unsupervised learning we have only input data
3,in supervised learning we have classification ...
4,in unsupervised learning we have clustering as...
5,delhi is captial of india


In [33]:
def generate_embeddings(text):
    embedding = openai.Embedding.create(input=text,model=embedding_model)['data'][0]['embedding']
    return embedding
    

In [34]:
## create new column embeddings and apply generate_embeddings function

df['embeddings'] = df['text'].apply(generate_embeddings)

In [35]:
df

,text,embeddings
0,machine learning has mainly two approaches sup...,"[-0.020156344398856163, 0.009695214219391346, ..."
1,in supervised learning we have input as well a...,"[-0.013303461484611034, -0.0002675261930562556..."
2,in unsupervised learning we have only input data,"[-0.018943047150969505, 0.010153364390134811, ..."
3,in supervised learning we have classification ...,"[0.003742062021046877, 0.011766345240175724, 0..."
4,in unsupervised learning we have clustering as...,"[-0.0031581269577145576, 0.01732719875872135, ..."
5,delhi is captial of india,"[-0.0008481526165269315, -0.004340545739978552..."


In [36]:
## convert embeddings column into numpy array
df['embeddings'] = df['embeddings'].apply(np.array)

In [39]:
h_df = Dataset.from_pandas(df)

In [40]:
h_df

Dataset({
    features: ['text', 'embeddings'],
    num_rows: 6
})

In [41]:
h_df.add_faiss_index(column='embeddings')

100%|██████████| 1/1 [00:00<00:00, 2438.55it/s]


Dataset({
    features: ['text', 'embeddings'],
    num_rows: 6
})

In [42]:
question = "what are types of machine learning"
question_embedding = generate_embeddings(question)
question_embedding = np.array(question_embedding)


In [51]:
scores, samples = h_df.get_nearest_examples(
    "embeddings", question_embedding, k=6
)

TypeError: handle_Index.<locals>.replacement_search() got an unexpected keyword argument 'distance_metric'

In [50]:
samples_df = pd.DataFrame.from_dict(samples)
samples_df["scores"] = scores
samples_df

,text,embeddings,scores
0,machine learning has mainly two approaches sup...,"[-0.020156344398856163, 0.009695214219391346, ...",0.249455
1,in supervised learning we have classification ...,"[0.003742062021046877, 0.011766345240175724, 0...",0.332448
2,in unsupervised learning we have clustering as...,"[-0.0031581269577145576, 0.01732719875872135, ...",0.394467
3,in supervised learning we have input as well a...,"[-0.013303461484611034, -0.0002675261930562556...",0.416249
4,in unsupervised learning we have only input data,"[-0.018943047150969505, 0.010153364390134811, ...",0.449413
5,delhi is captial of india,"[-0.0008481526165269315, -0.004340545739978552...",0.701531


In [47]:
samples

{'text': ['machine learning has mainly two approaches supervised learning as well as unsupervised learning',
  'in supervised learning we have classification as well as regression',
  'in unsupervised learning we have clustering as well as association'],
 'embeddings': [[-0.020156344398856163,
   0.009695214219391346,
   0.030713213607668877,
   -0.01970956102013588,
   -0.005556079093366861,
   0.01678631640970707,
   -0.009580327197909355,
   0.009510118514299393,
   -0.02179029770195484,
   -0.02076907642185688,
   -0.003382793627679348,
   0.01595657505095005,
   -0.01531831081956625,
   0.00786339957267046,
   0.006379438564181328,
   0.01865004375576973,
   0.017220335081219673,
   0.0013140238588675857,
   0.014807701110839844,
   -0.0022243463899940252,
   -0.021994542330503464,
   0.023066824302077293,
   0.006516664754599333,
   -0.04922284185886383,
   -0.006976214237511158,
   0.009471822530031204,
   0.0034689591266214848,
   -0.028415467590093613,
   -0.01568850316107273,

In [46]:
scores

array([0.24945472, 0.33244824, 0.39446688], dtype=float32)

In [13]:
tmp = generate_embeddings('ML is a subset of AI')
len(tmp),tmp[0:5]

(1536,
 [-0.011772715486586094,
  -0.006874183192849159,
  -0.0023190220817923546,
  -0.011447951197624207,
  -0.023856664076447487])